In [2]:
using JuMP, Clp

### 1. Polyhedron modeling
a) From the graph, the decision variables can be denoted as x,y,z.
$$\begin{aligned}
\qquad& x \le 1\\
\qquad& x \ge -1\\
\qquad& y \le 1\\
\qquad& y \ge -1\\
\qquad& z \le 1\\
\qquad& z \ge -1\\
\end{aligned}$$
Transform them into standard form:
$$\begin{aligned}
\qquad& x \le 1\\
\qquad& -x \le 1\\
\qquad& y \le 1\\
\qquad& -y \le 1\\
\qquad& z \le 1\\
\qquad& -z \le 1\\
\end{aligned}$$                                   
Therefore, matrix A is: 

In [3]:
[1 0 0;-1 0 0;0 1 0;0 -1 0;0 0 1;0 0 -1;]
                                    

6×3 Array{Int64,2}:
  1   0   0
 -1   0   0
  0   1   0
  0  -1   0
  0   0   1
  0   0  -1

b is:

In [4]:
[1;1;1;1;1;1]

6-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1

b) From the graph, the decision variables can be denoted as x,y,z.
$$\begin{aligned}
\qquad& x + y + z\le 1\\
\qquad& x - y + z\le 1\\
\qquad& -x + y + z\le 1\\
\qquad& -x - y + z\le 1\\
\qquad& x + y + z\ge -1\\
\qquad& x - y + z\ge -1\\
\qquad& -x + y + z\ge -1\\
\qquad& -x - y + z\ge -1\\
\end{aligned}$$
Transform them into standard form:
$$\begin{aligned}
\qquad& x + y + z\le 1\\
\qquad& x - y + z\le 1\\
\qquad& -x + y + z\le 1\\
\qquad& -x - y + z\le 1\\
\qquad& -x - y - z\le 1\\
\qquad& -x + y - z\le 1\\
\qquad& x - y - z\le 1\\
\qquad& x + y - z\le 1\\
\end{aligned}$$                                   
Therefore, matrix A is: 

In [5]:
[1 1 1; 1 -1 1; -1 1 1; -1 -1 1; -1 -1 -1; -1 1 -1; 1 -1 -1; 1 1 -1]

8×3 Array{Int64,2}:
  1   1   1
  1  -1   1
 -1   1   1
 -1  -1   1
 -1  -1  -1
 -1   1  -1
  1  -1  -1
  1   1  -1

b is

In [6]:
[1;1;1;1;1;1;1;1]

8-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1

### 2. Standard form with equality constraints
a) According to the transfomation tricks that we learned in class, inequalities can be transformed into equalities by adding slack. And in order to get the minimization instead of maximization, we can take the negative of it. Along with these two tricks, I can also shift the variables in order to convert bounded to nonnegative, as specified in the question.

For the bounded variables: <br>
let y2 = z2 + 1 so $-1 \le z_2 \le 5$ can be conterted into $y_2 + x_2 = 6$ such that $x_2, y_2 \ge 0$ <br>
let y3 = z3 + 1 so $-1 \le z_3 \le 5$ can be conterted into $y_3 + x_3 = 6$ such that $x_3, y_3 \ge 0$ <br>
let y4 = z4 + 2 so $-2 \le z_4 \le 2$ can be conterted into $y_4 + x_4 = 4$ such that $x_4, y_4 \ge 0$ <br>
For the inequalities: <br>
Replace the old variables first:<br>
$z_3 + z_4 \le 2$ is transformed to $y_3 + y_4 \le 5$ <br>
$7z_2 + z_4 = 5$ is transformed to $7y_2 + y_4 = 14$ <br>
For $-x_1 + 6z_2 - z_3 + z_4 \ge -3$, since $x_1$ is unbounded here, we set up variables u and v such that $u - v = x_1, u,v \ge 0$ <br>
Thus the inequality is transformed to $-(u - v) + 6y_2 - y_3 + y_4 \ge 4$ <br>
Next, transform the inequalities into equalities. <br>
$y_3 + y_4 \le 5$ is transformed to $y_3 + y_4 + s_1 = 5, s_1\ge0$<br>
$-(u - v) + 6y_2 - y_3 + y_4 \ge 4$ is transformed to $-(u - v) + 6y_2 - y_3 + y_4  - s_2 = 4, s_2\ge0$<br>
Then our objective is to minimize $-(3z_1 - z_2)$ which is $-(3(u-v) - y_2) = -3u + 3v + y_2$ <br>
(note $z_2 = y_2 -1$, so the solution satisfies (original) = -(new-1))


In [7]:
# we defined the vector x to be: 
#x = [u v y_2 y_3 y_4 x_2 x_3 x_4 s1 s2]
A = [0 0 1 0 0 1 0 0 0 0;0 0 0 1 0 0 1 0 0 0;0 0 0 0 1 0 0 1 0 0; -1 1 6 -1 1 0 0 0 0 -1; 0 0 7 0 1 0 0 0 0 0; 0 0 0 1 1 0 0 0 1 0]

6×10 Array{Int64,2}:
  0  0  1   0  0  1  0  0  0   0
  0  0  0   1  0  0  1  0  0   0
  0  0  0   0  1  0  0  1  0   0
 -1  1  6  -1  1  0  0  0  0  -1
  0  0  7   0  1  0  0  0  0   0
  0  0  0   1  1  0  0  0  1   0

In [8]:
b = [6; 6; 4; 4; 14; 5]

6-element Array{Int64,1}:
  6
  6
  4
  4
 14
  5

In [9]:
c = [-3;3;1;0;0;0;0;0;0;0]

10-element Array{Int64,1}:
 -3
  3
  1
  0
  0
  0
  0
  0
  0
  0

Relations of decision variables with those of origional LP:<br>
$z_1 = u-v, z_2 = y_2-1, z_3 = y_3-1,z_4 = y_4 -2$

b) Original problem:

In [10]:
original = Model(solver=ClpSolver())
@variable(original, z1)
@variable(original, -1<= z2 <= 5)
@variable(original, -1 <= z3 <= 5)
@variable(original, -2 <= z4 <= 2)

@constraint(original, -z1 + 6z2 - z3 + z4 >= -3)
@constraint(original, 7z2 + z4 == 5)
@constraint(original, z3 + z4 <= 2)
@objective(original, Max, 3z1 - z2 )

original

Maximization problem with:
 * 3 linear constraints
 * 4 variables
Solver is Clp

In [11]:
# transformed version
transform = Model(solver=ClpSolver())
@variable(transform, x[1:10] >= 0 )      # specify a vector variable x
@constraint(transform, A*x .== b )      # the dot in front of <=, 
                                      # which indicates element-wise (vector) inequalities
@objective(transform, Min, c'*x )       # Note the transpose of the cost vector c

transform

Minimization problem with:
 * 6 linear constraints
 * 10 variables
Solver is Clp

In [12]:
status = solve(original)

println(original)
println(status)
println()
println("z1 = ", getvalue(z1))
println("z2 = ", getvalue(z2))
println("z3 = ", getvalue(z3))
println("z4 = ", getvalue(z4))
println("objective = ", getobjectivevalue(original) )
println()
println()

Max 3 z1 - z2
Subject to
 -z1 + 6 z2 - z3 + z4 ≥ -3
 7 z2 + z4 = 5
 z3 + z4 ≤ 2
 z1
 -1 ≤ z2 ≤ 5
 -1 ≤ z3 ≤ 5
 -2 ≤ z4 ≤ 2

Optimal

z1 = 8.571428571428571
z2 = 0.42857142857142855
z3 = -1.0
z4 = 2.0
objective = 25.28571428571429




In [13]:
status = solve(transform)

println(transform)
println(status)
println()
println("z1 = ", getvalue(x[1]-x[2]))
println("z2 = ", getvalue(x[3] - 1) )
println("z3 = ", getvalue(x[4] - 1) )
println("z4 = ", getvalue(x[5] - 2) )
println("objective = ", -(getobjectivevalue(transform)) + 1)

Min -3 x[1] + 3 x[2] + x[3]
Subject to
 x[3] + x[6] = 6
 x[4] + x[7] = 6
 x[5] + x[8] = 4
 -x[1] + x[2] + 6 x[3] - x[4] + x[5] - x[10] = 4
 7 x[3] + x[5] = 14
 x[4] + x[5] + x[9] = 5
 x[i] ≥ 0 ∀ i ∈ {1,2,…,9,10}

Optimal

z1 = 8.571428571428571
z2 = 0.4285714285714286
z3 = -1.0
z4 = 2.0
objective = 25.28571428571429


### 3. Road Lighting

a) Our goal is to minimize the total power usage along the road. So we want to minimize $ \sum_{j = 1}^{m} p_j$, subject to $\sum_{j = 1}^{m} a_{ij} p_j \ge I_i^{min}$, with variable $p_j, j \in n$. <br>
This formulation is bounded by the constraint minimum required illumination of each road segment, as well as the condition that $p_j \ge 0$.<br>


b) Since the formulation does not have any upperbound, if we try to maximize the use of power, it's not going to provide with any solution, because it's unbounded.

c) Since applying strict inequality will prevent obtaining the optimal solution, and valid linear program formulation would provide us with the optimal solution. Therefure, I would not get the same solution as part a) once applied strict inequality.<br>
This is not a standard linear program.

### 4. Stigler's diet


In [14]:
# a)
# STARTER CODE FOR STIGLER'S DIET PROBLEM
using CSV
using JuMP, Clp
# import Stigler's data set
raw = CSV.read("stigler.csv")
# Get data

# list of food
foods = raw[2:end,1]

# list of nutrients
nutrients = [string(names(raw)[i]) for i=2:length(names(raw))]

# minimum required amount of each nutrient
lower_bound = raw[1,2:end]

# data[f,i] is the amount of nutrient i contained in food f 
data = raw[2:end,2:end]

A = (convert(Matrix, data))
b = convert(Matrix, lower_bound) * 365
c = ones(77)
compact = Model(solver=ClpSolver())
@variable(compact, x[1:77] >= 0 )      # specify a vector variable x
@constraint(compact, A'*x .>= b' )      # the dot in front of >=, 
                                      # which indicates element-wise (vector) inequalities
@objective(compact, Min, c'*x )       # Note the transpose of the cost vector c

status = solve(compact)


MethodError: MethodError: Cannot `convert` an object of type DataFrames.DataFrameRow{DataFrames.DataFrame,DataFrames.SubIndex{DataFrames.Index,UnitRange{Int64},UnitRange{Int64}}} to an object of type Array{T,2} where T
Closest candidates are:
  convert(::Type{T<:Array}, !Matched::AbstractArray) where T<:Array at array.jl:489
  convert(::Type{T<:AbstractArray}, !Matched::T<:AbstractArray) where T<:AbstractArray at abstractarray.jl:14
  convert(::Type{T<:AbstractArray}, !Matched::LinearAlgebra.Factorization) where T<:AbstractArray at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/LinearAlgebra/src/factorization.jl:46
  ...

In [15]:

println("The lowest cost is ", getobjectivevalue(compact))
println();
for i in 1:77
    if getvalue(x[i]) > 0
        println(foods[i])
            end
end
println()
println()

UndefVarError: UndefVarError: compact not defined

In [16]:

# STARTER CODE FOR STIGLER'S DIET PROBLEM
using CSV
using JuMP, Clp
# import Stigler's data set
raw = CSV.read("stigler_vege.csv")
# Get data

# list of food
foods = raw[2:end,1]

# list of nutrients
nutrients = [string(names(raw)[i]) for i=2:length(names(raw))]

# minimum required amount of each nutrient
lower_bound = raw[1,2:end]

# data[f,i] is the amount of nutrient i contained in food f 
data = raw[2:end,2:end]

A = (convert(Matrix, data))
b = convert(Matrix, lower_bound) * 365
c = ones(60)
compact = Model(solver=ClpSolver())
@variable(compact, x[1:60] >= 0 )      # specify a vector variable x
@constraint(compact, A'*x .>= b' )      # the dot in front of >=, 
                                      # which indicates element-wise (vector) inequalities
@objective(compact, Min, c'*x )       # Note the transpose of the cost vector c

status = solve(compact)


MethodError: MethodError: Cannot `convert` an object of type DataFrames.DataFrameRow{DataFrames.DataFrame,DataFrames.SubIndex{DataFrames.Index,UnitRange{Int64},UnitRange{Int64}}} to an object of type Array{T,2} where T
Closest candidates are:
  convert(::Type{T<:Array}, !Matched::AbstractArray) where T<:Array at array.jl:489
  convert(::Type{T<:AbstractArray}, !Matched::T<:AbstractArray) where T<:AbstractArray at abstractarray.jl:14
  convert(::Type{T<:AbstractArray}, !Matched::LinearAlgebra.Factorization) where T<:AbstractArray at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/LinearAlgebra/src/factorization.jl:46
  ...

In [17]:

println("The lowest cost is ", getobjectivevalue(compact))
println();
for i in 1:60
    if getvalue(x[i]) > 0
        println(foods[i])
            end
end
println()
println()

UndefVarError: UndefVarError: compact not defined